In [30]:
# Step 1: Load the earthquake JSON file from S3

import boto3
import json

# Initialize S3 client (ensure credentials are set with `aws configure`)
s3 = boto3.client('s3')

# Define the bucket and object key (path to your file)
bucket = 'azyryano-earthquake-pipeline'
key = 'raw/earthquake-data-2025-07-26T03-02-33Z.json'

try:
    # Retrieve the object from S3
    obj = s3.get_object(Bucket=bucket, Key=key)
    
    # Read and parse the JSON content
    data = json.loads(obj['Body'].read())

    # Inspect the data structure
    features = data.get('features', [])
    print("✅ Loaded JSON from S3")
    print(f"Total records found: {len(features)}")
    print("First record sample:")
    print(features[0])  # Preview to verify the structure

except Exception as e:
    print("❌ Failed to load JSON from S3:", str(e))


✅ Loaded JSON from S3
Total records found: 13
First record sample:
{'type': 'Feature', 'properties': {'mag': 1, 'place': '20 km N of Stanton, Texas', 'time': 1753498697122, 'updated': 1753498836508, 'tz': None, 'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/tx2025oojmyr', 'detail': 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/detail/tx2025oojmyr.geojson', 'felt': None, 'cdi': None, 'mmi': None, 'alert': None, 'status': 'automatic', 'tsunami': 0, 'sig': 15, 'net': 'tx', 'code': '2025oojmyr', 'ids': ',tx2025oojmyr,', 'sources': ',tx,', 'types': ',origin,phase-data,', 'nst': 24, 'dmin': 0, 'rms': 0.4, 'gap': 62, 'magType': 'ml', 'type': 'earthquake', 'title': 'M 1.0 - 20 km N of Stanton, Texas'}, 'geometry': {'type': 'Point', 'coordinates': [-101.789, 32.312, 4.9025]}, 'id': 'tx2025oojmyr'}


In [31]:
# Step 2: Parse the JSON into a structured pandas DataFrame

import pandas as pd

try:
    # Create a list of dictionaries for each earthquake record
    rows = []
    for f in features:  # Limit to 500 for speed/testing
        props = f.get('properties', {})
        rows.append({
            'id': f.get('id'),
            'place': props.get('place'),
            'magnitude': props.get('mag'),
            'time': int(props.get('time')) if props.get('time') else None
        })

    # Create DataFrame
    df = pd.DataFrame(rows)

    # Preview the DataFrame
    print("✅ DataFrame created")
    print(df.head())

except Exception as e:
    print("❌ Failed to create DataFrame:", str(e))


✅ DataFrame created
             id                                    place  magnitude  \
0  tx2025oojmyr                20 km N of Stanton, Texas       1.00   
1  ak0259ids12k         19 km E of Susitna North, Alaska       1.40   
2  tx2025oojjtw                   9 km W of Rotan, Texas       2.30   
3  ak0259idq82w  35 km S of Denali National Park, Alaska       1.90   
4    nc75215122                 6 km E of Santa Rosa, CA       1.86   

            time  
0  1753498697122  
1  1753498669564  
2  1753498476822  
3  1753498147282  
4  1753497521800  


In [32]:
# Step 3: Write DataFrame to an in-memory CSV buffer

import io

try:
    csv_buffer = io.StringIO()
    
    # Write to CSV with no header and no index
    df.to_csv(csv_buffer, index=False, header=False)
    
    # Reset buffer position to beginning so it can be read
    csv_buffer.seek(0)

    # Preview first few lines to ensure formatting is correct
    print("✅ CSV in memory preview:")
    for _ in range(5):
        print(csv_buffer.readline())
    csv_buffer.seek(0)  # Reset again for the next reader

except Exception as e:
    print("❌ Failed to write CSV to buffer:", str(e))


✅ CSV in memory preview:
tx2025oojmyr,"20 km N of Stanton, Texas",1.0,1753498697122

ak0259ids12k,"19 km E of Susitna North, Alaska",1.4,1753498669564

tx2025oojjtw,"9 km W of Rotan, Texas",2.3,1753498476822

ak0259idq82w,"35 km S of Denali National Park, Alaska",1.9,1753498147282

nc75215122,"6 km E of Santa Rosa, CA",1.86,1753497521800



In [33]:
# Step 4: Connect to RDS PostgreSQL instance using psycopg2

import psycopg2

try:
    conn = psycopg2.connect(
        host="database-azyryano.cfkcqqqsm3g5.us-east-2.rds.amazonaws.com",
        database="databaseazyryano",
        user="postgres",
        password="Your password",  # Replace with your real password
        port=5432
    )
    cur = conn.cursor()
    print("✅ Connected to PostgreSQL")

except Exception as e:
    print("❌ Failed to connect to PostgreSQL:", str(e))


✅ Connected to PostgreSQL


In [34]:
# Step 5: Use COPY command to insert all rows into PostgreSQL efficiently


try:
    copy_sql = """
        COPY earthquakes (id, place, magnitude, time)
        FROM STDIN WITH CSV;
    """
    print("⏳ Starting COPY operation...")
    # Clear the table first (optional, for full refreshes)
    cur.execute("TRUNCATE TABLE earthquakes;")

    cur.copy_expert(sql=copy_sql, file=csv_buffer)
    conn.commit()
    print("✅ Data successfully copied to database")

    # Optional: Validate insert
    cur.execute("SELECT COUNT(*) FROM earthquakes;")
    count = cur.fetchone()[0]
    print(f"📊 Total records in 'earthquakes' table: {count}")

    # Clean up
    cur.close()
    conn.close()

except Exception as e:
    print("❌ Failed to copy data to PostgreSQL:", str(e))


⏳ Starting COPY operation...
✅ Data successfully copied to database
📊 Total records in 'earthquakes' table: 13
